In [ ]:

# Financial Plan Summary Application
# ============================================================================
# INSTALLATION: Run this cell first to install all dependencies
# ============================================================================
!pip install langchain langchain-openai langchain-core langgraph openai plotly numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9


### 🤖 Integrate MCP with Claude Agent

Now let's enhance our financial planning agent to use these MCP servers for real-time data.

**What's happening:**
1. Claude can now call Yahoo Finance tools to get current market data
2. Claude can call FRED tools to get economic indicators
3. The agent automatically decides when to use these tools based on the query
4. All tool calls are handled through the MCP protocol

In [ ]:
# Install MCP SDK and server packages
!pip install -q mcp anthropic
!pip install -q yfinance fredapi

print("✅ MCP packages installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.9/405.9 kB 7.6 MB/s eta 0:00:00
✅ MCP packages installed successfully!


---
## 🔌 MCP Integration Setup

**What is MCP (Model Context Protocol)?**

MCP is a protocol that allows AI models to interact with external data sources and tools in a standardized way. Think of it as a universal adapter that lets Claude connect to various services like Yahoo Finance, FRED, databases, etc.

**Why MCP for this workshop?**
- ✅ Simple stdio-based communication (works great in Colab)
- ✅ Standardized interface across different data sources
- ✅ Real-time data access without manual API coding
- ✅ Perfect for agentic AI applications

We'll integrate two MCP servers:
1. **Yahoo Finance MCP** - Stock prices, portfolio data, market indices
2. **FRED MCP** - Economic indicators (inflation, unemployment, GDP, etc.)

### 📊 Yahoo Finance MCP Server

This MCP server provides access to:
- Real-time stock prices
- Historical market data
- Portfolio performance tracking
- Market indices (S&P 500, Dow Jones, etc.)

**How it works:** The server runs as a subprocess and communicates via JSON-RPC over stdio.

In [ ]:
import subprocess
import json
import yfinance as yf
from typing import Dict, List, Any

class YahooFinanceMCP:
    """Simplified Yahoo Finance MCP Server for Workshop"""

    def __init__(self):
        self.tools = [
            {
                "name": "get_stock_price",
                "description": "Get current stock price and basic info",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "symbol": {"type": "string", "description": "Stock ticker symbol (e.g., AAPL, TSLA)"}
                    },
                    "required": ["symbol"]
                }
            },
            {
                "name": "get_market_indices",
                "description": "Get major market indices (S&P 500, Dow Jones, Nasdaq)",
                "inputSchema": {"type": "object", "properties": {}}
            },
            {
                "name": "get_portfolio_value",
                "description": "Calculate total portfolio value from list of holdings",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "holdings": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "symbol": {"type": "string"},
                                    "shares": {"type": "number"}
                                }
                            }
                        }
                    },
                    "required": ["holdings"]
                }
            }
        ]

    def get_stock_price(self, symbol: str) -> Dict[str, Any]:
        """Fetch current stock price and info"""
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            return {
                "symbol": symbol.upper(),
                "price": info.get('currentPrice', info.get('regularMarketPrice', 'N/A')),
                "name": info.get('shortName', symbol),
                "change": info.get('regularMarketChange', 'N/A'),
                "change_percent": info.get('regularMarketChangePercent', 'N/A'),
                "market_cap": info.get('marketCap', 'N/A')
            }
        except Exception as e:
            return {"error": str(e)}

    def get_market_indices(self) -> Dict[str, Any]:
        """Fetch major market indices"""
        indices = {
            "S&P 500": "^GSPC",
            "Dow Jones": "^DJI",
            "Nasdaq": "^IXIC"
        }

        results = {}
        for name, symbol in indices.items():
            ticker = yf.Ticker(symbol)
            info = ticker.info
            results[name] = {
                "value": info.get('regularMarketPrice', 'N/A'),
                "change": info.get('regularMarketChange', 'N/A'),
                "change_percent": info.get('regularMarketChangePercent', 'N/A')
            }
        return results

    def get_portfolio_value(self, holdings: List[Dict]) -> Dict[str, Any]:
        """Calculate total portfolio value"""
        total_value = 0
        positions = []

        for holding in holdings:
            symbol = holding['symbol']
            shares = holding['shares']

            ticker = yf.Ticker(symbol)
            price = ticker.info.get('currentPrice', ticker.info.get('regularMarketPrice', 0))
            value = price * shares
            total_value += value

            positions.append({
                "symbol": symbol,
                "shares": shares,
                "price": price,
                "value": value
            })

        return {
            "total_value": total_value,
            "positions": positions,
            "count": len(positions)
        }

    def call_tool(self, tool_name: str, arguments: Dict) -> Any:
        """Execute tool call"""
        if tool_name == "get_stock_price":
            return self.get_stock_price(arguments['symbol'])
        elif tool_name == "get_market_indices":
            return self.get_market_indices()
        elif tool_name == "get_portfolio_value":
            return self.get_portfolio_value(arguments['holdings'])
        else:
            return {"error": f"Unknown tool: {tool_name}"}

# Initialize Yahoo Finance MCP
yahoo_mcp = YahooFinanceMCP()
print("✅ Yahoo Finance MCP server initialized")
print(f"   Available tools: {[tool['name'] for tool in yahoo_mcp.tools]}")

✅ Yahoo Finance MCP server initialized
   Available tools: ['get_stock_price', 'get_market_indices', 'get_portfolio_value']


### 🏛️ FRED (Federal Reserve Economic Data) MCP Server

Access to Federal Reserve economic data:
- Inflation rates (CPI, PCE)
- Unemployment statistics
- Federal Funds Rate
- Mortgage rates
- GDP data

**Note:** FRED requires an API key (free). Get one at: https://fred.stlouisfed.org/docs/api/api_key.html

For this workshop, we'll use sample data if no API key is provided.

In [ ]:
from fredapi import Fred
from datetime import datetime, timedelta
import pandas as pd

class FREDMCP:
    """Simplified FRED MCP Server for Workshop"""

    def __init__(self, api_key: str = None):
        self.api_key = api_key
        self.fred = Fred(api_key=api_key) if api_key else None

        # Common FRED series IDs
        self.series_map = {
            "inflation": "CPIAUCSL",  # Consumer Price Index
            "unemployment": "UNRATE",  # Unemployment Rate
            "fed_funds": "FEDFUNDS",   # Federal Funds Rate
            "mortgage_30y": "MORTGAGE30US",  # 30-Year Mortgage Rate
            "gdp": "GDP",  # Gross Domestic Product
            "gdp_growth": "A191RL1Q225SBEA"  # Real GDP Growth Rate
        }

        self.tools = [
            {
                "name": "get_economic_indicator",
                "description": "Get economic indicator data (inflation, unemployment, fed_funds, mortgage_30y, gdp, gdp_growth)",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "indicator": {
                            "type": "string",
                            "description": "Indicator name: inflation, unemployment, fed_funds, mortgage_30y, gdp, gdp_growth"
                        },
                        "months_back": {
                            "type": "integer",
                            "description": "Number of months of historical data (default: 12)"
                        }
                    },
                    "required": ["indicator"]
                }
            },
            {
                "name": "get_all_indicators",
                "description": "Get current values for all major economic indicators",
                "inputSchema": {"type": "object", "properties": {}}
            }
        ]

    def get_economic_indicator(self, indicator: str, months_back: int = 12) -> Dict[str, Any]:
        """Fetch economic indicator data"""
        if not self.fred:
            return self._get_sample_data(indicator, months_back)

        try:
            series_id = self.series_map.get(indicator)
            if not series_id:
                return {"error": f"Unknown indicator: {indicator}. Available: {list(self.series_map.keys())}"}

            end_date = datetime.now()
            start_date = end_date - timedelta(days=months_back * 30)

            data = self.fred.get_series(series_id, start_date, end_date)

            previous_year_value = None
            if indicator == 'inflation' and len(data) >= 13: # Special handling for year-over-year inflation
                previous_year_value = float(data.iloc[-13])
            elif len(data) > 1: # For other indicators or if not enough data for year-over-year
                previous_year_value = float(data.iloc[-2])

            return {
                "indicator": indicator,
                "series_id": series_id,
                "current_value": float(data.iloc[-1]) if len(data) > 0 else None,
                "previous_value": previous_year_value,
                "change": float(data.iloc[-1] - previous_year_value) if previous_year_value is not None else None,
                "data_points": len(data),
                "last_updated": str(data.index[-1]) if len(data) > 0 else None
            }
        except Exception as e:
            return {"error": str(e)}

    def get_all_indicators(self) -> Dict[str, Any]:
        """Get all major economic indicators"""
        results = {}
        for indicator in self.series_map.keys():
            results[indicator] = self.get_economic_indicator(indicator, months_back=1)
        return results

    def _get_sample_data(self, indicator: str, months_back: int) -> Dict[str, Any]:
        """Return sample data when no API key is provided"""
        sample_values = {
            "inflation": 3.2,
            "unemployment": 3.7,
            "fed_funds": 5.33,
            "mortgage_30y": 7.22,
            "gdp": 27000,  # billions
            "gdp_growth": 2.4
        }

        return {
            "indicator": indicator,
            "current_value": sample_values.get(indicator, 0),
            "note": "Sample data - provide FRED API key for real-time data",
            "data_points": months_back
        }

    def call_tool(self, tool_name: str, arguments: Dict) -> Any:
        """Execute tool call"""
        if tool_name == "get_economic_indicator":
            return self.get_economic_indicator(
                arguments['indicator'],
                arguments.get('months_back', 12)
            )
        elif tool_name == "get_all_indicators":
            return self.get_all_indicators()
        else:
            return {"error": f"Unknown tool: {tool_name}"}

# Initialize FRED MCP (with user prompt for API key)
fred_api_key_input = input("Enter your FRED API Key (press Enter to use sample data): ").strip()
fred_api_key = fred_api_key_input if fred_api_key_input else None

fred_mcp = FREDMCP(api_key=fred_api_key)
print("✅ FRED MCP server initialized")
print(f"   Available tools: {[tool['name'] for tool in fred_mcp.tools]}")
if fred_api_key:
    print("   💡 Using real-time data from FRED")
else:
    print("   💡 Using sample data (provide API key for real-time data)")

Enter your FRED API Key (press Enter to use sample data): 70343004909d838025cc776ee15976d0
✅ FRED MCP server initialized
   Available tools: ['get_economic_indicator', 'get_all_indicators']
   💡 Using real-time data from FRED


### 🧪 Test the MCP Servers

Let's verify both MCP servers are working correctly by making some test calls.

In [ ]:
print("=" * 60)
print("Testing Yahoo Finance MCP")
print("=" * 60)

# Test 1: Get stock price
print("\n1️⃣ Getting AAPL stock price...")
result = yahoo_mcp.call_tool("get_stock_price", {"symbol": "AAPL"})
print(json.dumps(result, indent=2))

# Test 2: Get market indices
print("\n2️⃣ Getting market indices...")
result = yahoo_mcp.call_tool("get_market_indices", {})
print(json.dumps(result, indent=2))

# Test 3: Calculate portfolio value
print("\n3️⃣ Calculating sample portfolio value...")
result = yahoo_mcp.call_tool("get_portfolio_value", {
    "holdings": [
        {"symbol": "AAPL", "shares": 10},
        {"symbol": "GOOGL", "shares": 5},
        {"symbol": "MSFT", "shares": 8}
    ]
})
print(json.dumps(result, indent=2))

print("\n" + "=" * 60)
print("Testing FRED MCP")
print("=" * 60)

# Test 4: Get unemployment rate
print("\n4️⃣ Getting unemployment rate...")
result = fred_mcp.call_tool("get_economic_indicator", {"indicator": "unemployment"})
print(json.dumps(result, indent=2))

# Test 5: Get all economic indicators
print("\n5️⃣ Getting all economic indicators...")
result = fred_mcp.call_tool("get_all_indicators", {})
print(json.dumps(result, indent=2))

print("\n✅ All MCP tests completed!")

Testing Yahoo Finance MCP

1️⃣ Getting AAPL stock price...
{
  "symbol": "AAPL",
  "price": 255.78,
  "name": "Apple Inc.",
  "change": -5.95001,
  "change_percent": -2.27334,
  "market_cap": 3759435415552
}

2️⃣ Getting market indices...
{
  "S&P 500": {
    "value": 6836.17,
    "change": 3.4101562,
    "change_percent": 0.049908914
  },
  "Dow Jones": {
    "value": 49500.93,
    "change": 48.94922,
    "change_percent": 0.09898333
  },
  "Nasdaq": {
    "value": 22546.67,
    "change": -50.478516,
    "change_percent": -0.22338445
  }
}

3️⃣ Calculating sample portfolio value...
{
  "total_value": 7296.960000000001,
  "positions": [
    {
      "symbol": "AAPL",
      "shares": 10,
      "price": 255.78,
      "value": 2557.8
    },
    {
      "symbol": "GOOGL",
      "shares": 5,
      "price": 305.72,
      "value": 1528.6000000000001
    },
    {
      "symbol": "MSFT",
      "shares": 8,
      "price": 401.32,
      "value": 3210.56
    }
  ],
  "count": 3
}

Testing FRED MCP


### 🛡️ Fallback Mechanism Demo

Let's demonstrate how the fallback mechanism works when MCP tools fail or return incomplete data.

**What happens:**
1. Agent attempts to fetch real-time data from MCP
2. If successful → uses live data ✅
3. If failed → automatically falls back to defaults 🔄
4. Application continues without crashing 💪

In [ ]:
print("="*60)
print("🧪 Testing Fallback Mechanism")
print("="*60)

# --- BEGIN EMBEDDED DEPENDENCIES FOR FALLBACK_DEMO ---
import json
from typing import Dict, List, Any

# Re-define MCPToolWrapper here for self-contained testing
# This is a temporary measure for demonstration purposes within this cell.
class MCPToolWrapper:
    """Wrapper for MCP tools with automatic fallback handling"""

    def __init__(self, yahoo_mcp, fred_mcp):
        self.yahoo_mcp = yahoo_mcp
        self.fred_mcp = fred_mcp
        self.fallback_values = {
            # FRED fallbacks
            'inflation': 3.2,
            'unemployment': 3.7,
            'fed_funds': 5.33,
            'mortgage_30y': 7.22,
            'gdp_growth': 2.4,
            # Market fallbacks
            'market_return': 0.08,  # 8% annual return
            'sp500_return': 0.10,   # 10% annual return
        }

    def get_inflation_rate(self) -> float:
        """Get current inflation rate with fallback"""
        try:
            result = self.fred_mcp.call_tool('get_economic_indicator', {
                'indicator': 'inflation',
                'months_back': 1
            })

            if 'error' not in result and result.get('current_value'):
                # FRED returns CPI, convert to inflation rate
                if result.get('previous_value'):
                    # Assuming CPIAUCSL is annual % change, which is not always the case for current_value vs prev_value
                    # For this demo, let's assume current_value is already the rate or base for a simple % change
                    # Simplified to use a fixed rate if no previous_value for accurate calculation in FRED
                    inflation = result['current_value'] # Simplified for demo, actual FRED CPI calculation is more complex
                    print(f"📊 Using real-time inflation rate: {inflation:.2f}%")
                    return inflation / 100  # Convert to decimal

            print(f"⚠️  MCP returned incomplete data, using fallback")
            return self._use_fallback('inflation')
        except Exception as e:
            print(f"⚠️  MCP call failed ({str(e)}), using fallback inflation rate")
            return self._use_fallback('inflation')

    def get_unemployment_rate(self) -> float:
        """Get current unemployment rate with fallback"""
        try:
            result = self.fred_mcp.call_tool('get_economic_indicator', {
                'indicator': 'unemployment',
                'months_back': 1
            })

            if 'error' not in result and result.get('current_value'):
                rate = result['current_value']
                print(f"📊 Using real-time unemployment rate: {rate:.2f}%")
                return rate / 100  # Convert to decimal

            return self._use_fallback('unemployment')
        except Exception as e:
            print(f"⚠️  MCP call failed, using fallback unemployment rate")
            return self._use_fallback('unemployment')

    def get_market_performance(self) -> Dict[str, float]:
        """Get current market indices with fallback"""
        try:
            result = self.yahoo_mcp.call_tool('get_market_indices', {})

            if 'error' not in result and result.get('S&P 500'):
                sp500_change = result['S&P 500'].get('change_percent', 0)
                print(f"📊 Using real-time market data: S&P 500 change {sp500_change:.2f}%")
                return {
                    'sp500_change': sp500_change / 100,
                    'market_status': 'live_data',
                    'indices': result
                }

            return self._use_market_fallback()
        except Exception as e:
            print(f"⚠️  MCP call failed, using fallback market data")
            return self._use_market_fallback()

    def get_stock_portfolio_value(self, holdings: List[Dict]) -> Dict[str, Any]:
        """Calculate portfolio value with fallback"""
        try:
            result = self.yahoo_mcp.call_tool('get_portfolio_value', {
                'holdings': holdings
            })

            if 'error' not in result and result.get('total_value'):
                print(f"📊 Using real-time portfolio valuation: ${result['total_value']:,.2f}")
                return result

            return self._use_portfolio_fallback(holdings)
        except Exception as e:
            print(f"⚠️  MCP call failed, using estimated portfolio value")
            return self._use_portfolio_fallback(holdings)

    def get_mortgage_rate(self) -> float:
        """Get current 30-year mortgage rate with fallback"""
        try:
            result = self.fred_mcp.call_tool('get_economic_indicator', {
                'indicator': 'mortgage_30y',
                'months_back': 1
            })

            if 'error' not in result and result.get('current_value'):
                rate = result['current_value']
                print(f"📊 Using real-time mortgage rate: {rate:.2f}%")
                return rate / 100

            return self._use_fallback('mortgage_30y')
        except Exception as e:
            print(f"⚠️  MCP call failed, using fallback mortgage rate")
            return self._use_fallback('mortgage_30y')

    def _use_fallback(self, key: str) -> float:
        """Use fallback value and log it"""
        value = self.fallback_values.get(key, 0)
        print(f"🔄 Using fallback {key}: {value * 100:.2f}%")
        return value

    def _use_market_fallback(self) -> Dict[str, Any]:
        """Use fallback market data"""
        print(f"🔄 Using fallback market return: {self.fallback_values['market_return'] * 100:.2f}%")
        return {
            'sp500_change': 0.0,
            'market_status': 'fallback_data',
            'annual_return_assumption': self.fallback_values['market_return']
        }

    def _use_portfolio_fallback(self, holdings: List[Dict]) -> Dict[str, Any]:
        """Estimate portfolio value using fallback prices"""
        # Use rough estimates: $150/share average
        total = sum(h['shares'] * 150 for h in holdings)
        print(f"🔄 Using estimated portfolio value: ${total:,.2f}")
        return {
            'total_value': total,
            'positions': [],
            'count': len(holdings),
            'status': 'estimated'
        }

# Re-define enhanced calculation tools for self-contained testing
# Assumes 'mcp_tools' is available globally or within this scope
def calculate_retirement_needs_mcp(user_info: Dict[str, Any]) -> Dict[str, Any]:
    """Calculate retirement needs using real-time inflation from FRED MCP"""

    # Get real-time or fallback inflation rate
    if mcp_tools:
        inflation_rate = mcp_tools.get_inflation_rate()
    else:
        inflation_rate = 0.032  # 3.2% fallback
        print(f"🔄 MCP not available, using default inflation: {inflation_rate * 100:.2f}%")

    current_age = user_info.get('age', 30)
    retirement_age = user_info.get('retirement_age', 65)
    annual_income = user_info.get('annual_income', 0)
    current_savings = user_info.get('savings', 0)
    life_expectancy = user_info.get('life_expectancy', 85)

    years_to_retirement = retirement_age - current_age
    years_in_retirement = life_expectancy - retirement_age

    # Use 70% of current income as retirement income need
    annual_retirement_expenses = annual_income * 0.7

    # Adjust for inflation using real-time rate
    future_annual_expenses = annual_retirement_expenses * ((1 + inflation_rate) ** years_to_retirement)

    # Calculate total needed (simplified)
    total_retirement_fund_needed = future_annual_expenses * years_in_retirement

    # Calculate monthly savings needed
    months_to_retirement = years_to_retirement * 12
    monthly_savings_needed = (total_retirement_fund_needed - current_savings) / months_to_retirement if months_to_retirement > 0 else 0

    return {
        "total_needed": total_retirement_fund_needed,
        "monthly_savings": monthly_savings_needed,
        "annual_retirement_expenses": future_annual_expenses,
        "years_to_retirement": years_to_retirement,
        "years_in_retirement": years_in_retirement,
        "inflation_rate_used": inflation_rate,
        "data_source": "MCP-FRED" if mcp_tools else "fallback"
    }

def calculate_wealth_allocation_mcp(user_info: Dict[str, Any]) -> Dict[str, Any]:
    """Calculate wealth allocation considering current market conditions from Yahoo Finance MCP"""

    # Get real-time market performance
    if mcp_tools:
        market_data = mcp_tools.get_market_performance()
    else:
        market_data = {'market_status': 'fallback_data', 'annual_return_assumption': 0.08}
        print(f"🔄 MCP not available, using default market assumptions")

    age = user_info.get('age', 30)
    risk_tolerance = user_info.get('risk_tolerance', 'moderate').lower()

    # Base allocation on age and risk tolerance
    stock_allocation = min(110 - age, 90) / 100

    # Adjust based on risk tolerance
    if risk_tolerance == 'aggressive':
        stock_allocation = min(stock_allocation + 0.15, 0.95)
    elif risk_tolerance == 'conservative':
        stock_allocation = max(stock_allocation - 0.15, 0.30)

    bond_allocation = 1.0 - stock_allocation

    return {
        "stocks": stock_allocation * 100,
        "bonds": bond_allocation * 100,
        "risk_level": risk_tolerance,
        "market_conditions": market_data.get('market_status', 'unknown'),
        "current_market_trend": market_data.get('sp500_change', 0) * 100,
        "data_source": "MCP-Yahoo" if market_data.get('market_status') == 'live_data' else "fallback"
    }
# --- END EMBEDDED DEPENDENCIES FOR FALLBACK_DEMO ---


# Initialize the MCP tool wrapper here to ensure it's defined
try:
    # Ensure yahoo_mcp and fred_mcp are available from prior cell executions
    mcp_tools = MCPToolWrapper(yahoo_mcp, fred_mcp)
    print("✅ MCP Tool Wrapper initialized with fallback support inside test cell")
except Exception as e:
    print(f"⚠️  Could not initialize MCP wrapper inside test cell: {e}")
    mcp_tools = None

# Test 1: Try to get inflation rate
print("\n1️⃣ Testing Inflation Rate Fetch...")
if mcp_tools:
    inflation = mcp_tools.get_inflation_rate()
    print(f"   Result: {inflation*100:.2f}%\n")
else:
    print("   ⚠️ MCP tools not initialized\n")

# Test 2: Try to get market data
print("2️⃣ Testing Market Data Fetch...")
if mcp_tools:
    market = mcp_tools.get_market_performance()
    print(f"   Status: {market['market_status']}")
    print(f"   Result: {json.dumps(market, indent=4)}\n")
else:
    print("   ⚠️ MCP tools not initialized\n")

# Test 3: Calculate retirement needs with MCP
print("3️⃣ Testing Retirement Calculation with Real-Time Data...")
test_user_info = {
    'age': 35,
    'retirement_age': 65,
    'annual_income': 100000,
    'savings': 50000,
    'life_expectancy': 85
}

# Ensure calculate_retirement_needs_mcp is callable
result = calculate_retirement_needs_mcp(test_user_info)
print(f"   Data source: {result['data_source']}")
print(f"   Inflation rate used: {result['inflation_rate_used']*100:.2f}%")
print(f"   Monthly savings needed: ${result['monthly_savings']:,.2f}")
print(f"   Total fund needed: ${result['total_needed']:,.2f}\n")

print("="*60)
print("✅ All tests completed - application never crashed!")
print("="*60)

🧪 Testing Fallback Mechanism
✅ MCP Tool Wrapper initialized with fallback support inside test cell

1️⃣ Testing Inflation Rate Fetch...
⚠️  MCP returned incomplete data, using fallback
🔄 Using fallback inflation: 320.00%
   Result: 320.00%

2️⃣ Testing Market Data Fetch...
📊 Using real-time market data: S&P 500 change 0.05%
   Status: live_data
   Result: {
    "sp500_change": 0.00049908914,
    "market_status": "live_data",
    "indices": {
        "S&P 500": {
            "value": 6836.17,
            "change": 3.4101562,
            "change_percent": 0.049908914
        },
        "Dow Jones": {
            "value": 49500.93,
            "change": 48.94922,
            "change_percent": 0.09898333
        },
        "Nasdaq": {
            "value": 22546.67,
            "change": -50.478516,
            "change_percent": -0.22338445
        }
    }
}

3️⃣ Testing Retirement Calculation with Real-Time Data...
⚠️  MCP returned incomplete data, using fallback
🔄 Using fallback inflation

---
## 🎓 Workshop Recap: MCP Integration

**What we learned:**

1. **MCP Protocol Basics**
   - Standardized way to connect AI agents to external tools
   - Works via stdio (perfect for Colab)
   - Each MCP server exposes tools with schemas

2. **Yahoo Finance MCP**
   - Real-time stock prices
   - Market indices tracking
   - Portfolio calculations

3. **FRED MCP**
   - Economic indicators (inflation, unemployment, etc.)
   - Federal Reserve data
   - Historical trends

**Next Steps:**
- Agent Integration
- Build more complex agentic workflows
- Deploy as a production service

**Resources:**
- MCP Docs: https://modelcontextprotocol.io/
- Anthropic Claude API: https://docs.anthropic.com/
- Yahoo Finance API: https://python-yahoofinance.readthedocs.io/
- FRED API: https://fred.stlouisfed.org/docs/api/

In [ ]:
# ============================================================================
# IMPORTS
# ============================================================================
import os
from typing import TypedDict, Annotated, Sequence, List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
import operator
from datetime import datetime
import json

# STEP 1: Set Your OpenAI API Key


In [ ]:

OPENAI_API_KEY = input("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# STEP 2: Define Shared Tools

This code block first prompts the user to enter their OpenAI API Key and then sets it as an environment variable. After that, it initializes the `ChatOpenAI` Large Language Model (LLM) with the specified model and temperature, which will be used by the agents throughout the application.

This code block first prompts the user to enter their OpenAI API Key and then sets it as an environment variable. After that, it initializes the `ChatOpenAI` Large Language Model (LLM) with the specified model and temperature, which will be used by the agents throughout the application.

This code block defines several helper functions, each representing a financial calculation tool. These tools are decorated with `@tool` from the `langchain` library, making them callable by the LLM-powered agents. The tools cover calculations for retirement needs, life insurance, education funds, estate tax, and wealth allocation.

In [ ]:
@tool
def calculate_retirement_needs(current_age: int, retirement_age: int,
                               annual_expenses: float, life_expectancy: int = 85) -> str:
    """Calculate estimated retirement fund needed based on age and expenses."""
    years_to_retirement = retirement_age - current_age
    years_in_retirement = life_expectancy - retirement_age

    # Simple calculation with 3% inflation
    inflation_rate = 0.03
    future_annual_expenses = annual_expenses * ((1 + inflation_rate) ** years_to_retirement)
    total_needed = future_annual_expenses * years_in_retirement

    return f"""Retirement Calculation:
- Years until retirement: {years_to_retirement}
- Years in retirement: {years_in_retirement}
- Future annual expenses (adjusted for inflation): ${future_annual_expenses:,.2f}
- Total retirement fund needed: ${total_needed:,.2f}
- Recommended monthly savings: ${(total_needed / (years_to_retirement * 12)):,.2f}"""

@tool
def calculate_life_insurance(annual_income: float, num_dependents: int,
                            outstanding_debts: float, savings: float) -> str:
    """Calculate recommended life insurance coverage."""
    # Rule of thumb: 10x annual income + debts - savings
    base_coverage = (annual_income * 10) + outstanding_debts - savings
    dependent_factor = 1 + (num_dependents * 0.2)
    recommended_coverage = base_coverage * dependent_factor

    return f"""Life Insurance Recommendation:
- Base coverage needed: ${base_coverage:,.2f}
- Adjustment for {num_dependents} dependent(s): {dependent_factor}x
- Recommended coverage: ${recommended_coverage:,.2f}
- Estimated monthly premium (term life): ${(recommended_coverage * 0.0005 / 12):,.2f}"""

@tool
def calculate_education_fund(num_children: int, children_ages: List[int],
                             cost_per_year: float = 30000) -> str:
    """Calculate education fund needed for children."""
    total_needed = 0
    details = []

    for i, age in enumerate(children_ages):
        years_until_college = max(0, 18 - age)
        years_in_college = 4

        # Inflation adjusted cost
        inflation_rate = 0.05
        future_cost = cost_per_year * ((1 + inflation_rate) ** years_until_college)
        child_total = future_cost * years_in_college
        total_needed += child_total

        details.append(f"Child {i+1} (age {age}): ${child_total:,.2f} needed in {years_until_college} years")

    return f"""Education Fund Calculation:
{chr(10).join(details)}
- Total education fund needed: ${total_needed:,.2f}
- Recommended monthly savings: ${(total_needed / (max([18 - age for age in children_ages if age < 18] + [1]) * 12)):,.2f}"""

@tool
def calculate_estate_tax(total_assets: float, state: str = "Federal") -> str:
    """Estimate potential estate taxes."""
    federal_exemption = 13610000  # 2024 exemption
    taxable_estate = max(0, total_assets - federal_exemption)
    estimated_tax = taxable_estate * 0.40  # Top federal rate

    return f"""Estate Tax Estimation:
- Total assets: ${total_assets:,.2f}
- Federal exemption: ${federal_exemption:,.2f}
- Taxable estate: ${taxable_estate:,.2f}
- Estimated federal estate tax: ${estimated_tax:,.2f}
- Recommendation: {'Estate planning strategies recommended' if taxable_estate > 0 else 'Below exemption threshold'}"""

@tool
def calculate_wealth_allocation(total_assets: float, age: int, risk_tolerance: str = "moderate") -> str:
    """Recommend asset allocation based on age and risk tolerance."""
    # Age-based stock allocation
    base_stock_pct = 100 - age

    # Adjust for risk tolerance
    if risk_tolerance.lower() == "aggressive":
        stock_pct = min(90, base_stock_pct + 10)
    elif risk_tolerance.lower() == "conservative":
        stock_pct = max(20, base_stock_pct - 20)
    else:
        stock_pct = base_stock_pct

    bond_pct = 100 - stock_pct

    stock_amount = total_assets * (stock_pct / 100)
    bond_amount = total_assets * (bond_pct / 100)

    return f"""Asset Allocation Recommendation:
- Risk Profile: {risk_tolerance.title()}
- Stocks/Equity: {stock_pct}% (${stock_amount:,.2f})
- Bonds/Fixed Income: {bond_pct}% (${bond_amount:,.2f})
- Rebalance: Quarterly or when allocation drifts 5%+"""

# Enhanced calculation tools that use MCP data
# ============================================================================

@tool
def calculate_retirement_needs_mcp(user_info: Dict[str, Any]) -> Dict[str, Any]:
    """Calculate retirement needs using real-time inflation from FRED MCP"""

    # Get real-time or fallback inflation rate
    if mcp_tools:
        inflation_rate = mcp_tools.get_inflation_rate()
    else:
        inflation_rate = 0.032  # 3.2% fallback
        print(f"🔄 MCP not available, using default inflation: {inflation_rate * 100:.2f}%")

    current_age = user_info.get('age', 30)
    retirement_age = user_info.get('retirement_age', 65)
    annual_income = user_info.get('annual_income', 0)
    current_savings = user_info.get('savings', 0)
    life_expectancy = user_info.get('life_expectancy', 85)

    years_to_retirement = retirement_age - current_age
    years_in_retirement = life_expectancy - retirement_age

    # Use 70% of current income as retirement income need
    annual_retirement_expenses = annual_income * 0.7

    # Adjust for inflation using real-time rate
    future_annual_expenses = annual_retirement_expenses * ((1 + inflation_rate) ** years_to_retirement)

    # Calculate total needed (simplified)
    total_retirement_fund_needed = future_annual_expenses * years_in_retirement

    # Calculate monthly savings needed
    months_to_retirement = years_to_retirement * 12
    monthly_savings_needed = (total_retirement_fund_needed - current_savings) / months_to_retirement if months_to_retirement > 0 else 0

    return {
        "total_needed": total_retirement_fund_needed,
        "monthly_savings": monthly_savings_needed,
        "annual_retirement_expenses": future_annual_expenses,
        "years_to_retirement": years_to_retirement,
        "years_in_retirement": years_in_retirement,
        "inflation_rate_used": inflation_rate,
        "data_source": "MCP-FRED" if mcp_tools else "fallback"
    }

@tool
def calculate_wealth_allocation_mcp(user_info: Dict[str, Any]) -> Dict[str, Any]:
    """Calculate wealth allocation considering current market conditions from Yahoo Finance MCP"""

    # Get real-time market performance
    if mcp_tools:
        market_data = mcp_tools.get_market_performance()
    else:
        market_data = {'market_status': 'fallback_data', 'annual_return_assumption': 0.08}
        print(f"🔄 MCP not available, using default market assumptions")

    age = user_info.get('age', 30)
    risk_tolerance = user_info.get('risk_tolerance', 'moderate').lower()

    # Base allocation on age and risk tolerance
    stock_allocation = min(110 - age, 90) / 100

    # Adjust based on risk tolerance
    if risk_tolerance == 'aggressive':
        stock_allocation = min(stock_allocation + 0.15, 0.95)
    elif risk_tolerance == 'conservative':
        stock_allocation = max(stock_allocation - 0.15, 0.30)

    bond_allocation = 1.0 - stock_allocation

    return {
        "stocks": stock_allocation * 100,
        "bonds": bond_allocation * 100,
        "risk_level": risk_tolerance,
        "market_conditions": market_data.get('market_status', 'unknown'),
        "current_market_trend": market_data.get('sp500_change', 0) * 100,
        "data_source": "MCP-Yahoo" if market_data.get('market_status') == 'live_data' else "fallback"
    }

print("✅ MCP-enhanced calculation tools ready")


@tool
def calculate_tax_optimization(annual_income: float, filing_status: str = "married",
                              retirement_contributions: float = 0, charitable_giving: float = 0) -> str:
    """Calculate tax optimization strategies and potential savings."""
    # 2024 tax brackets (simplified)
    if filing_status.lower() == "married":
        standard_deduction = 29200
        brackets = [(22275, 0.10), (89450, 0.12), (190750, 0.22), (364200, 0.24), (462500, 0.32), (693750, 0.35)]
    else:
        standard_deduction = 14600
        brackets = [(11000, 0.10), (44725, 0.12), (95375, 0.22), (182050, 0.24), (231250, 0.32), (578125, 0.35)]

    # Calculate current tax
    taxable_income = max(0, annual_income - standard_deduction - retirement_contributions)
    current_tax = 0
    remaining_income = taxable_income

    for bracket_max, rate in brackets:
        if remaining_income > 0:
            taxable_at_bracket = min(remaining_income, bracket_max)
            current_tax += taxable_at_bracket * rate
            remaining_income -= taxable_at_bracket

    # Tax optimization strategies
    max_401k = 23000  # 2024 limit
    max_ira = 7000    # 2024 limit
    additional_retirement = max_401k - retirement_contributions

    # Calculate tax savings from additional contributions
    marginal_rate = 0.22  # Approximate marginal rate for middle income
    retirement_tax_savings = min(additional_retirement, annual_income * 0.15) * marginal_rate


    return f"""Tax Optimization Analysis:
- Current taxable income: ${taxable_income:,.0f}
- Estimated current tax: ${current_tax:,.0f}
- Marginal tax rate: {marginal_rate:.0%}

Optimization Opportunities:
- Additional 401(k) contributions: ${additional_retirement:,.0f} (saves ${retirement_tax_savings:,.0f})
- IRA contribution potential: ${max_ira:,.0f} (saves ${max_ira * marginal_rate:,.0f})
- HSA max contribution: $4,300 (saves ${4300 * marginal_rate:,.0f})

Recommended: Maximize pre-tax retirement contributions and consider tax-loss harvesting"""


@tool
def calculate_529_plan(num_children: int, children_ages: List[int],
                      annual_contribution: float = 0, state: str = "National") -> str:
    """Calculate 529 education savings plan projections and benefits."""
    total_needed = 0
    total_savings = 0
    details = []

    for i, age in enumerate(children_ages):
        years_until_college = max(0, 18 - age)

        # Future college cost (assuming 5% annual increase)
        current_cost = 35000  # Average annual college cost
        future_cost = current_cost * ((1.05) ** years_until_college)
        four_year_total = future_cost * 4
        total_needed += four_year_total

        # 529 savings projection (assuming 6% annual return)
        if years_until_college > 0:
            monthly_contribution = annual_contribution / 12 / num_children
            future_value = monthly_contribution * (((1 + 0.06/12) ** (years_until_college * 12) - 1) / (0.06/12))
            total_savings += future_value

            details.append(f"Child {i+1} (age {age}): ${four_year_total:,.0f} needed, ${future_value:,.0f} projected savings")
        else:
            details.append(f"Child {i+1} (age {age}): Currently in college - ${four_year_total:,.0f} needed")

    shortfall = max(0, total_needed - total_savings)

    return f"""529 Education Savings Analysis:
{chr(10).join(details)}
- Total education costs projected: ${total_needed:,.0f}
- Projected 529 savings: ${total_savings:,.0f}
- Funding gap: ${shortfall:,.0f}
- Tax benefits: State deduction varies by state
- Recommended: {'Increase contributions' if shortfall > 0 else 'On track for funding'}"""


@tool
def calculate_education_fund(num_children: int, children_ages: List[int],
                             cost_per_year: float = 30000) -> str:
    """Calculate education fund needed for children."""
    total_needed = 0
    details = []

    for i, age in enumerate(children_ages):
        years_until_college = max(0, 18 - age)
        years_in_college = 4

        # Inflation adjusted cost
        inflation_rate = 0.05
        future_cost = cost_per_year * ((1 + inflation_rate) ** years_until_college)
        child_total = future_cost * years_in_college
        total_needed += child_total

        details.append(f"Child {i+1} (age {age}): ${child_total:,.2f} needed in {years_until_college} years")

    # Guard against division by zero
    months_until_college = max([18 - age for age in children_ages if age < 18] + [1]) * 12
    monthly_savings = 0 if months_until_college <= 0 else total_needed / months_until_college

    return f"""Education Fund Calculation:
{chr(10).join(details)}
- Total education fund needed: ${total_needed:,.2f}
- Recommended monthly savings: ${monthly_savings:,.2f}"""


@tool
def analyze_scholarship_opportunities(student_age: int, gpa: float = 0.0,
                                    family_income: float = 0, activities: str = "") -> str:
    """Analyze scholarship opportunities and financial aid eligibility."""
    opportunities = []

    # Merit-based scholarships
    if gpa >= 3.8:
        opportunities.append("High academic achievement scholarships (up to $20,000/year)")
    elif gpa >= 3.5:
        opportunities.append("Academic merit scholarships (up to $10,000/year)")
    elif gpa >= 3.0:
        opportunities.append("Merit-based awards (up to $5,000/year)")

    # Need-based aid
    if family_income < 60000:
        opportunities.append("Maximum Pell Grant eligibility (~$7,400/year)")
        opportunities.append("State need-based grants")
    elif family_income < 100000:
        opportunities.append("Partial need-based aid eligibility")

    # Activity-based
    if "sports" in activities.lower():
        opportunities.append("Athletic scholarships (varies by sport and division)")
    if "music" in activities.lower() or "art" in activities.lower():
        opportunities.append("Arts and performance scholarships")
    if "volunteer" in activities.lower() or "community" in activities.lower():
        opportunities.append("Community service scholarships")

    # Other opportunities
    opportunities.extend([
        "Local community foundation scholarships",
        "Professional association scholarships",
        "Employer tuition assistance programs"
    ])

    return f"""Scholarship and Financial Aid Analysis:
Student Profile: Age {student_age}, GPA {gpa:.1f}, Family Income ${family_income:,.0f}

Eligible Opportunities:
{chr(10).join([f'- {opp}' for opp in opportunities])}

Next Steps:
- Complete FAFSA for need-based aid
- Research school-specific scholarships
- Apply to external scholarship databases
- Consider community college transfer option
- Explore work-study programs"""


@tool
def project_retirement_inflation(years: int) -> float:
    """Project inflation rate over a given number of years using real-time data or fallback."""
    if mcp_tools:
        inflation_rate = mcp_tools.get_inflation_rate()
        print(f"📊 Using real-time inflation rate for projection: {inflation_rate * 100:.2f}%")
    else:
        inflation_rate = 0.032  # 3.2% fallback
        print(f"🔄 MCP not available, using default inflation for projection: {inflation_rate * 100:.2f}%")

    # Simple projection: assume inflation rate remains constant
    projected_inflation_factor = (1 + inflation_rate) ** years

    return projected_inflation_factor


@tool
def get_inflation_rate() -> float:
    """Get current inflation rate from MCP, with fallback."""
    if mcp_tools:
        inflation_rate = mcp_tools.get_inflation_rate()
        print(f"📊 Current inflation rate from MCP: {inflation_rate * 100:.2f}%")
        return inflation_rate
    else:
        inflation_rate = 0.032  # 3.2% fallback
        print(f"🔄 MCP not available, using default inflation: {inflation_rate * 100:.2f}%")
        return inflation_rate


@tool
def get_federal_funds_rate() -> float:
    """Get current federal funds rate from MCP, with fallback."""
    if mcp_tools:
        fed_funds_rate = mcp_tools.get_fed_funds_rate()
        print(f"📊 Current federal funds rate from MCP: {fed_funds_rate * 100:.2f}%")
        return fed_funds_rate
    else:
        fed_funds_rate = 0.0533  # 5.33% fallback
        print(f"🔄 MCP not available, using default federal funds rate: {fed_funds_rate * 100:.2f}%")
        return fed_funds_rate

✅ MCP-enhanced calculation tools ready


# STEP 3: Define State for Agent Graph
The AgentState in this notebook is a TypedDict that defines the structure of the state object passed between different agents in the financial planning application. It holds all the necessary information for the agents to process and build the financial plan. Here's a breakdown of its components:

* messages: A sequence of HumanMessage, AIMessage, or SystemMessage objects, representing the conversation history. It accumulates messages as the planning progresses.

* user_info: A dictionary containing all the personal and financial information provided by the user (e.g., age, income, savings, risk tolerance, number of dependents, etc.).

* selected_plans: A list of strings indicating which financial planning areas the user has chosen (e.g., 'Retirement Planning', 'Insurance Planning').

* plan_summaries: A dictionary where keys are the names of the planning areas and values are the generated summaries for each area. This is where the output of individual agents is stored.

* next_agent: A string indicating which agent should be executed next in a more complex graph flow, though in this specific setup, the OrchestratorAgent explicitly calls the other agents.

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage | AIMessage | SystemMessage], operator.add]
    user_info: Dict[str, Any]
    selected_plans: List[str]
    plan_summaries: Dict[str, str]
    next_agent: str
    mcp_data: Dict[str, Any]

---
## 🔄 MCP-Enhanced Agent Tools with Fallback

**Strategy:**
- Agents will attempt to use real-time MCP data first
- If MCP calls fail, agents automatically fall back to default values
- Application never crashes due to external data failures

**Enhanced Agents:**
1. **Retirement Agent** - Uses FRED for real-time inflation rates
2. **Wealth Agent** - Uses Yahoo Finance for portfolio analysis
3. **Insurance Agent** - Uses FRED for economic context
4. **Estate Agent** - Uses market data for asset valuation
5. **Education Agent** - To be implemented as part of the workshop
6. **Tax Agent** - To be implemented as part of the workshop

In [ ]:
# MCP Tool Wrapper with Fallback
# ============================================================================
# This wrapper ensures agents never crash when MCP tools are unavailable
# ============================================================================

class MCPToolWrapper:
    """Wrapper for MCP tools with automatic fallback handling"""

    def __init__(self, yahoo_mcp, fred_mcp):
        self.yahoo_mcp = yahoo_mcp
        self.fred_mcp = fred_mcp
        self.fallback_values = {
            # FRED fallbacks (now as decimal rates)
            'inflation': 0.032,  # 3.2%
            'unemployment': 0.037, # 3.7%
            'fed_funds': 0.0533,   # 5.33%
            'mortgage_30y': 0.0722,  # 7.22%
            'gdp': 27000, # Billions (not a percentage)
            'gdp_growth': 0.024, # 2.4%
            # Market fallbacks
            'market_return': 0.08,  # 8% annual return
            'sp500_return': 0.10,   # 10% annual return
        }

    def get_inflation_rate(self) -> float:
        """Get current inflation rate with fallback"""
        try:
            # Request 13 months of data to calculate year-over-year inflation
            result = self.fred_mcp.call_tool('get_economic_indicator', {
                'indicator': 'inflation',
                'months_back': 13
            })

            if 'error' not in result and result.get('current_value') and result.get('previous_value'):
                # FRED returns CPI, calculate year-over-year inflation rate
                # The 'previous_value' from FRED here would be from 12 months ago if months_back=13 is handled correctly by FREDMCP
                inflation = ((result['current_value'] - result['previous_value']) / result['previous_value']) * 100
                print(f"📊 Using real-time year-over-year inflation rate: {inflation:.2f}%")
                return inflation / 100  # Convert to decimal
            elif 'error' not in result and result.get('current_value'):
                # This case should be less frequent with months_back=13 for CPIAUCSL
                print(f"⚠️  MCP returned partial data for inflation, interpreting current_value as rate: {result['current_value']:.2f}%")
                return result['current_value'] / 100 # Convert to decimal

            print(f"⚠️  MCP returned incomplete data for inflation, using fallback")
            return self._use_fallback('inflation')
        except Exception as e:
            print(f"⚠️  MCP call failed for inflation ({str(e)}), using fallback inflation rate")
            return self._use_fallback('inflation')

    def get_unemployment_rate(self) -> float:
        """Get current unemployment rate with fallback"""
        try:
            result = self.fred_mcp.call_tool('get_economic_indicator', {
                'indicator': 'unemployment',
                'months_back': 1
            })

            if 'error' not in result and result.get('current_value'):
                rate = result['current_value']
                print(f"📊 Using real-time unemployment rate: {rate:.2f}%")
                return rate / 100  # Convert to decimal

            return self._use_fallback('unemployment')
        except Exception as e:
            print(f"⚠️  MCP call failed, using fallback unemployment rate")
            return self._use_fallback('unemployment')

    def get_market_performance(self) -> Dict[str, float]:
        """Get current market indices with fallback"""
        try:
            result = self.yahoo_mcp.call_tool('get_market_indices', {})

            if 'error' not in result and result.get('S&P 500'):
                sp500_change = result['S&P 500'].get('change_percent', 0)
                print(f"📊 Using real-time market data: S&P 500 change {sp500_change:.2f}%")
                return {
                    'sp500_change': sp500_change / 100,
                    'market_status': 'live_data',
                    'indices': result
                }

            return self._use_market_fallback()
        except Exception as e:
            print(f"⚠️  MCP call failed, using fallback market data")
            return self._use_market_fallback()

    def get_stock_portfolio_value(self, holdings: List[Dict]) -> Dict[str, Any]:
        """Calculate portfolio value with fallback"""
        try:
            result = self.yahoo_mcp.call_tool('get_portfolio_value', {
                'holdings': holdings
            })

            if 'error' not in result and result.get('total_value'):
                print(f"📊 Using real-time portfolio valuation: ${result['total_value']:,.2f}")
                return result

            return self._use_portfolio_fallback(holdings)
        except Exception as e:
            print(f"⚠️  MCP call failed, using estimated portfolio value")
            return self._use_portfolio_fallback(holdings)

    def get_mortgage_rate(self) -> float:
        """Get current 30-year mortgage rate with fallback"""
        try:
            result = self.fred_mcp.call_tool('get_economic_indicator', {
                'indicator': 'mortgage_30y',
                'months_back': 1
            })

            if 'error' not in result and result.get('current_value'):
                rate = result['current_value']
                print(f"📊 Using real-time mortgage rate: {rate:.2f}%")
                return rate / 100

            return self._use_fallback('mortgage_30y')
        except Exception as e:
            print(f"⚠️  MCP call failed, using fallback mortgage rate")
            return self._use_fallback('mortgage_30y')

    def _use_fallback(self, key: str) -> float:
        """Use fallback value and log it"""
        value = self.fallback_values.get(key, 0)
        # Log percentages as percentages, other values directly
        if key in ['inflation', 'unemployment', 'fed_funds', 'mortgage_30y', 'gdp_growth', 'market_return', 'sp500_return']:
            print(f"🔄 Using fallback {key}: {value * 100:.2f}%")
        else:
            print(f"🔄 Using fallback {key}: {value}")
        return value

    def _use_market_fallback(self) -> Dict[str, Any]:
        """Use fallback market data"""
        print(f"🔄 Using fallback market return: {self.fallback_values['market_return'] * 100:.2f}%")
        return {
            'sp500_change': 0.0,
            'market_status': 'fallback_data',
            'annual_return_assumption': self.fallback_values['market_return']
        }

    def _use_portfolio_fallback(self, holdings: List[Dict]) -> Dict[str, Any]:
        """Estimate portfolio value using fallback prices"""
        # Use rough estimates: $150/share average
        total = sum(h['shares'] * 150 for h in holdings)
        print(f"🔄 Using estimated portfolio value: ${total:,.2f}")
        return {
            'total_value': total,
            'positions': [],
            'count': len(holdings),
            'status': 'estimated'
        }

# Initialize the MCP tool wrapper
try:
    mcp_tools = MCPToolWrapper(yahoo_mcp, fred_mcp)
    print("✅ MCP Tool Wrapper initialized with fallback support")
except Exception as e:
    print(f"⚠️  Could not initialize MCP wrapper: {e}")
    mcp_tools = None

✅ MCP Tool Wrapper initialized with fallback support


# STEP 4: Create Specialized Agents
Defines four specialized agent classes: RetirementAgent, InsuranceAgent, EstateAgent, WealthAgent, EducationAgent and TaxAgent.

Each of these agents is designed to handle a specific financial planning domain:

* Initialization (__init__): Each agent is initialized with the llm (Large
Language Model) and a list of tools relevant to its domain. For example, RetirementAgent uses calculate_retirement_needs_mcp and calculate_wealth_allocation.
* Processing (process): This is the core method for each agent. It takes the current AgentState as input.
  * It constructs a detailed prompt for the LLM based on the user_info from the state.
  * It binds the agent's specific tools to the LLM.
  * It invokes the LLM with the prompt. The LLM then decides whether to call any of the bound tools.
  * If the LLM generates tool_calls, the agent executes these tools and uses their results to generate a more comprehensive summary.
  * Finally, it updates the plan_summaries dictionary in the AgentState with the generated summary for its respective planning area and adds a confirmation message to the messages list.

In [ ]:
class RetirementAgent:
    def __init__(self, llm):
        self.llm = llm
        self.tools = [calculate_retirement_needs_mcp, calculate_wealth_allocation_mcp]

    def process(self, state: AgentState) -> AgentState:
        user_info = state["user_info"]

        prompt = f"""You are a Retirement Planning Specialist with access to real-time economic data via MCP. Analyze the user's information and create a comprehensive retirement plan.

User Information:
- Age: {user_info.get('age', 'Not provided')}
- Retirement Age: {user_info.get('retirement_age', 'Not provided')}
- Annual Income: ${user_info.get('annual_income', 0):,.2f}
- Current Savings: ${user_info.get('savings', 0):,.2f}
- Risk Tolerance: {user_info.get('risk_tolerance', 'moderate')}

Use the available tools to calculate retirement needs and recommend asset allocation.
Provide a detailed retirement plan summary."""

        tools_to_use = self.tools
        llm_with_tools = self.llm.bind_tools(tools_to_use)

        response = llm_with_tools.invoke([HumanMessage(content=prompt)])

        # Execute tool calls if any
        if response.tool_calls:
            tool_results = []
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                tool_args = tool_call["args"]

                # Find and execute the tool
                tool_func = None
                for tool in tools_to_use:
                    if tool.name == tool_name:
                        tool_func = tool
                        break

                if tool_func:
                    # Explicitly pass user_info for MCP-enhanced tools that expect it
                    if tool_name in ["calculate_retirement_needs_mcp", "calculate_wealth_allocation_mcp"]:
                        # For these tools, the 'user_info' dict from the agent's state is the primary argument
                        result = tool_func.invoke({"user_info": user_info})
                    else:
                        # For other tools, use the arguments as provided by the LLM
                        result = tool_func.invoke(tool_args)
                    # Convert dictionary results to JSON string for proper concatenation
                    tool_results.append(json.dumps(result, indent=2))
                else:
                    tool_results.append(json.dumps({"error": f"Unknown tool: {tool_name}"}, indent=2))

            # Create final summary with tool results
            summary_prompt = f"""Based on these calculations:
{chr(10).join(tool_results)}

Create a comprehensive retirement planning summary with specific recommendations."""

            final_response = self.llm.invoke([HumanMessage(content=summary_prompt)])
            summary = final_response.content
        else:
            summary = response.content

        state["plan_summaries"]["Retirement Planning"] = summary
        state["messages"].append(AIMessage(content=f"✓ Retirement Planning Complete"))

        return state

class InsuranceAgent:
    def __init__(self, llm):
        self.llm = llm
        self.tools = [calculate_life_insurance]

    def process(self, state: AgentState) -> AgentState:
        user_info = state["user_info"]

        prompt = f"""You are an Insurance Planning Specialist. Analyze the user's situation and recommend appropriate insurance coverage.

User Information:
- Age: {user_info.get('age', 'Not provided')}
- Annual Income: ${user_info.get('annual_income', 0):,.2f}
- Number of Dependents: {user_info.get('num_dependents', 0)}
- Current Savings: ${user_info.get('savings', 0):,.2f}
- Outstanding Debts: ${user_info.get('debts', 0):,.2f}

Use the life insurance calculation tool and provide comprehensive insurance recommendations
including life, health, disability, and liability insurance."""

        llm_with_tools = self.llm.bind_tools(self.tools)
        response = llm_with_tools.invoke([HumanMessage(content=prompt)])

        if response.tool_calls:
            tool_results = []
            for tool_call in response.tool_calls:
                result = calculate_life_insurance.invoke(tool_call["args"])
                tool_results.append(result)

            summary_prompt = f"""Based on these calculations:
{chr(10).join(tool_results)}

Create a comprehensive insurance planning summary covering life, health, disability, and liability insurance."""

            final_response = self.llm.invoke([HumanMessage(content=summary_prompt)])
            summary = final_response.content
        else:
            summary = response.content

        state["plan_summaries"]["Insurance Planning"] = summary
        state["messages"].append(AIMessage(content=f"✓ Insurance Planning Complete"))

        return state

class EstateAgent:
    def __init__(self, llm):
        self.llm = llm
        self.tools = [calculate_estate_tax, calculate_education_fund]

    def process(self, state: AgentState) -> AgentState:
        user_info = state["user_info"]

        prompt = f"""You are an Estate Planning Specialist. Create a comprehensive estate plan.

User Information:
- Age: {user_info.get('age', 'Not provided')}
- Total Assets: ${user_info.get('total_assets', user_info.get('savings', 0)):,.2f}
- Number of Children: {user_info.get('num_children', 0)}
- Children Ages: {user_info.get('children_ages', [])}

Use the available tools to calculate estate taxes and education funding needs.
Provide recommendations for wills, trusts, beneficiaries, and legacy planning."""

        llm_with_tools = self.llm.bind_tools(self.tools)
        response = llm_with_tools.invoke([HumanMessage(content=prompt)])

        if response.tool_calls:
            tool_results = []
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                for tool in self.tools:
                    if tool.name == tool_name:
                        result = tool.invoke(tool_call["args"])
                        tool_results.append(result)

            summary_prompt = f"""Based on these calculations:
{chr(10).join(tool_results)}

Create a comprehensive estate planning summary including wills, trusts, beneficiary designations,
education funding, and tax minimization strategies."""

            final_response = self.llm.invoke([HumanMessage(content=summary_prompt)])
            summary = final_response.content
        else:
            summary = response.content

        state["plan_summaries"]["Estate Planning"] = summary
        state["messages"].append(AIMessage(content=f"✓ Estate Planning Complete"))

        return state

class WealthAgent:
    def __init__(self, llm):
        self.llm = llm
        self.tools = [calculate_wealth_allocation]

    def process(self, state: AgentState) -> AgentState:
        user_info = state["user_info"]

        prompt = f"""You are a Personal Wealth Management Specialist. Create a comprehensive wealth management strategy.

User Information:
- Age: {user_info.get('age', 'Not provided')}
- Annual Income: ${user_info.get('annual_income', 0):,.2f}
- Total Assets: ${user_info.get('total_assets', user_info.get('savings', 0)):,.2f}
- Risk Tolerance: {user_info.get('risk_tolerance', 'moderate')}

Use the asset allocation tool and provide recommendations for:
- Investment strategy
- Tax optimization
- Cash flow management
- Diversification
- Regular portfolio review schedule"""

        llm_with_tools = self.llm.bind_tools(self.tools)
        response = llm_with_tools.invoke([HumanMessage(content=prompt)])

        if response.tool_calls:
            tool_results = []
            for tool_call in response.tool_calls:
                result = calculate_wealth_allocation.invoke(tool_call["args"])
                tool_results.append(result)

            summary_prompt = f"""Based on these calculations:
{chr(10).join(tool_results)}

Create a comprehensive wealth management summary including investment strategy, tax optimization,
diversification recommendations, and monitoring schedule."""

            final_response = self.llm.invoke([HumanMessage(content=summary_prompt)])
            summary = final_response.content
        else:
            summary = response.content

        state["plan_summaries"]["Personal Wealth Management"] = summary
        state["messages"].append(AIMessage(content=f"✓ Wealth Management Planning Complete"))

        return state


class EducationAgent:
    """Education Planning Specialist Agent

    Helps clients plan and save for educational expenses. It explores options for tuition funding,
    researches scholarships, and develops loan strategies based on individual needs and timelines.
    Whether it's 529 plans or other education savings vehicles, this agent knows the options.
    """
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm
        # self.tools = [calculate_education_fund, calculate_529_plan, analyze_scholarship_opportunities, project_retirement_inflation]

    def process(self, state: AgentState) -> AgentState:
        user_info = state["user_info"]

        prompt = f"""You are an Education Planning Specialist who helps clients plan and save for
educational expenses. You explore options for tuition funding, research scholarships, and develop
loan strategies based on individual needs and timelines. You're an expert in 529 plans and other
education savings vehicles.

User Information:
- Number of Children: {user_info.get('num_children', 0)}
- Children Ages: {user_info.get('children_ages', [])}
- Annual Income: ${user_info.get('annual_income', 0):,.2f}
- Current Education Savings: ${user_info.get('education_savings', 0):,.2f}
- Annual Education Contribution: ${user_info.get('annual_education_contribution', 0):,.2f}

IMPORTANT: You MUST use the available tools to provide accurate education planning:

1. Call calculate_education_fund with number of children and their ages to determine funding needs
2. Call calculate_529_plan to show tax-advantaged saving strategies
3. Call analyze_scholarship_opportunities to identify funding sources
4. Call project_retirement_inflation to understand cost escalation over time

After using these tools, provide comprehensive education planning recommendations including:
- Total education funding needs by child
- 529 plan strategy and contribution recommendations
- Scholarship opportunities and research guidance
- Loan optimization approaches
- Timeline-based funding strategy"""

# TODO
        state["plan_summaries"]["Education Planning"] = "Not Implemented"
        state["messages"].append(AIMessage(content="✓ Education Planning Needs to be implemented"))

        return state


class TaxAgent:
    """Tax Planning Specialist Agent

    Optimizes tax strategies, identifies deductions, and assists with compliance to minimize
    liabilities and maximize savings year-round. This is particularly valuable because tax
    implications touch almost every financial decision. This agent ensures we're making
    tax-efficient choices across the entire financial plan.
    """
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm
        # self.tools = [calculate_tax_optimization, get_inflation_rate, get_federal_funds_rate]

    def process(self, state: AgentState) -> AgentState:
        user_info = state["user_info"]

        prompt = f"""You are a Tax Planning Specialist who optimizes tax strategies, identifies
deductions, and assists with compliance to minimize liabilities and maximize savings year-round.
Tax implications touch almost every financial decision, so you ensure tax-efficient choices
across the entire financial plan.

User Information:
- Annual Income: ${user_info.get('annual_income', 0):,.2f}
- Filing Status: {user_info.get('filing_status', 'married')}
- Current Retirement Contributions: ${user_info.get('retirement_contributions', 0):,.2f}
- Charitable Giving: ${user_info.get('charitable_giving', 0):,.2f}
- Other Deductions: ${user_info.get('other_deductions', 0):,.2f}

IMPORTANT: You MUST use the available tools to provide accurate tax planning:

1. Call calculate_tax_optimization with income and deductions to identify tax savings opportunities
2. Call get_inflation_rate to understand impact on tax brackets and deductions
3. Call get_federal_funds_rate to assess interest rate implications for tax strategy

After using these tools, provide comprehensive tax planning strategies including:
- Deduction optimization and maximization
- Retirement contribution strategies (401k, IRA, backdoor conversions)
- Tax-loss harvesting opportunities
- Estimated quarterly tax planning
- Year-round tax-efficient decision making"""

# TODO
        state["plan_summaries"]["Tax Planning"] = "Not implemented"
        state["messages"].append(AIMessage(content="✓ Tax Planning Needs Implementation"))

        return state

# STEP 5: Orchestrator Agent
The code block in Step 5 defines the OrchestratorAgent class, which is responsible for coordinating the execution of the specialized financial planning agents and generating an integrated summary.

Here's a breakdown of its components:

* Initialization (__init__): The OrchestratorAgent is initialized with the llm (Large Language Model) and instances of each specialized agent (RetirementAgent, InsuranceAgent, EstateAgent, WealthAgent). This allows it to call upon these individual agents to perform their specific tasks.

* route method: This method takes the current AgentState as input and orchestrates the workflow. It iterates through the selected_plans in the AgentState and, for each selected plan that hasn't been summarized yet, it calls the process method of the corresponding specialized agent. After all relevant individual plans have been processed, it then calls the create_integrated_summary method to consolidate the results.

* create_integrated_summary method: This method is responsible for taking all the individual plan summaries generated by the specialized agents, along with the user_info, and using the LLM to synthesize them into a single, cohesive "Executive Summary." This summary provides an overview, highlights key recommendations, identifies synergies, and proposes an action plan.

In [ ]:
class OrchestratorAgent:
    def __init__(self, llm):
        self.llm = llm
        self.retirement_agent = RetirementAgent(llm)
        self.insurance_agent = InsuranceAgent(llm)
        self.estate_agent = EstateAgent(llm)
        self.wealth_agent = WealthAgent(llm)
        self.education_agent = EducationAgent(llm)
        self.tax_agent = TaxAgent(llm)

    def route(self, state: AgentState) -> AgentState:
        """Route to appropriate agents based on selected plans"""
        selected = state["selected_plans"]

        if "Retirement Planning" in selected and "Retirement Planning" not in state["plan_summaries"]:
            state = self.retirement_agent.process(state)

        if "Insurance Planning" in selected and "Insurance Planning" not in state["plan_summaries"]:
            state = self.insurance_agent.process(state)

        if "Estate Planning" in selected and "Estate Planning" not in state["plan_summaries"]:
            state = self.estate_agent.process(state)

        if "Personal Wealth Management" in selected and "Personal Wealth Management" not in state["plan_summaries"]:
            state = self.wealth_agent.process(state)

        if "Education Planning" in selected and "Education Planning" not in state["plan_summaries"]:
            state = self.education_agent.process(state)

        if "Tax Planning" in selected and "Tax Planning" not in state["plan_summaries"]:
            state = self.tax_agent.process(state)

        # Generate integrated summary
        state = self.create_integrated_summary(state)

        return state

    def create_integrated_summary(self, state: AgentState) -> AgentState:
        """Create an integrated summary of all plans"""
        summaries = state["plan_summaries"]
        user_info = state["user_info"]

        prompt = f"""You are a Senior Financial Advisor. Create an integrated Executive Summary
that combines all the individual plan summaries into a cohesive financial plan.

User Profile:
{json.dumps(user_info, indent=2)}

Individual Plan Summaries:
{chr(10).join([f"### {plan}:{chr(10)}{summary}{chr(10)}" for plan, summary in summaries.items()])}

Create an Executive Summary that:
0. Do not process the any of these request if you see 'Not Implemented', just return the message this is your homework
1. Provides an overview of the client's financial situation
2. Highlights key recommendations from each planning area
3. Identifies synergies and priorities across plans
4. Provides a clear action plan with timeline
5. Notes any areas requiring immediate attention"""

        response = self.llm.invoke([HumanMessage(content=prompt)])
        state["plan_summaries"]["Executive Summary"] = response.content

        return state

# STEP 6: Main Application
The code block in Step 6 defines the main application logic for the financial plan summary, primarily through two functions: collect_user_info and run_planning_application.

* collect_user_info(selected_plans: List[str]) -> Dict[str, Any]:

  * Purpose: This function is responsible for interactively gathering specific financial and personal details from the user based on the planning areas they selected. It ensures that only relevant information (e.g., retirement age if 'Retirement Planning' is chosen) is requested.
  * Process: It prompts the user for inputs like age, annual income, savings, and then conditionally asks for more specific details related to chosen plans (e.g., desired retirement age, number of dependents, children's ages, risk tolerance).
  * Output: It returns a dictionary (info) containing all the collected user data.

* run_planning_application():

  * Purpose: This is the central function that orchestrates the entire financial planning process from start to finish.
  * Process:
    1. Plan Selection: It first presents the user with a list of available financial planning options (Retirement, Insurance, Estate, Wealth Management) and allows them to select which plans they want.
    2. User Information Collection: It then calls collect_user_info to gather the necessary data from the user based on their selections.
    3. State Initialization: An initial AgentState is created, populated with the collected user_info and selected_plans.
    4. Orchestration: It instantiates the OrchestratorAgent and calls its route method, passing the initial state. This triggers the specialized agents to generate their respective plan summaries and the integrated executive summary.
    5. Display Results: Finally, it prints out the comprehensive financial plan, starting with the Executive Summary and then presenting each individual plan summary.

In [ ]:
def collect_user_info(selected_plans: List[str]) -> Dict[str, Any]:
    """Collect user information based on selected plans"""
    info = {}

    print("\n" + "="*60)
    print("PLEASE PROVIDE YOUR INFORMATION")
    print("="*60)

    # Basic info needed for all plans
    info['age'] = int(input("Your current age: "))
    info['annual_income'] = float(input("Annual income ($): "))
    info['savings'] = float(input("Current savings/assets ($): "))

    if "Retirement Planning" in selected_plans:
        info['retirement_age'] = int(input("Desired retirement age: "))
        risk = input("Risk tolerance (conservative/moderate/aggressive): ").lower()
        info['risk_tolerance'] = risk if risk in ['conservative', 'moderate', 'aggressive'] else 'moderate'

    if "Insurance Planning" in selected_plans:
        info['num_dependents'] = int(input("Number of dependents: "))
        info['debts'] = float(input("Outstanding debts ($): "))

    if "Estate Planning" in selected_plans:
        info['num_children'] = int(input("Number of children: "))
        if info['num_children'] > 0:
            ages = input(f"Ages of children (comma-separated): ")
            info['children_ages'] = [int(age.strip()) for age in ages.split(',')]
        else:
            info['children_ages'] = []
        info['total_assets'] = info['savings']

    if "Personal Wealth Management" in selected_plans:
        info['total_assets'] = info['savings']
        if 'risk_tolerance' not in info:
            risk = input("Risk tolerance (conservative/moderate/aggressive): ").lower()
            info['risk_tolerance'] = risk if risk in ['conservative', 'moderate', 'aggressive'] else 'moderate'

    return info

def run_planning_application():
    """Main application runner"""
    print("\n" + "="*60)
    print("FINANCIAL PLAN SUMMARY APPLICATION")
    print("="*60)

    # Plan selection
    available_plans = [
        "Retirement Planning",
        "Insurance Planning",
        "Estate Planning",
        "Personal Wealth Management",
        "Education Planning",
        "Tax Planning"
    ]

    print("\nAvailable Planning Options:")
    for i, plan in enumerate(available_plans, 1):
        print(f"{i}. {plan}")

    selection = input("\nEnter plan numbers (comma-separated, e.g., 1,2,3,4,5,6): ")
    selected_indices = [int(x.strip()) - 1 for x in selection.split(',')]
    selected_plans = [available_plans[i] for i in selected_indices if 0 <= i < len(available_plans)]

    if not selected_plans:
        print("No valid plans selected. Exiting.")
        return None

    print(f"\nSelected Plans: {', '.join(selected_plans)}")

    # Collect user information
    user_info = collect_user_info(selected_plans)

    # Initialize state
    initial_state = AgentState(
        messages=[],
        user_info=user_info,
        selected_plans=selected_plans,
        plan_summaries={},
        next_agent="",
        mcp_data={}
    )

    # Run orchestrator
    print("\n" + "="*60)
    print("GENERATING YOUR FINANCIAL PLAN...")
    print("="*60 + "\n")

    orchestrator = OrchestratorAgent(llm)
    final_state = orchestrator.route(initial_state)

    # Display results
    print("\n" + "="*60)
    print("YOUR COMPREHENSIVE FINANCIAL PLAN")
    print("="*60 + "\n")

    # Show executive summary first
    if "Executive Summary" in final_state["plan_summaries"]:
        print("EXECUTIVE SUMMARY")
        print("-" * 60)
        print(final_state["plan_summaries"]["Executive Summary"])
        print("\n")

    # Show individual plans
    for plan, summary in final_state["plan_summaries"].items():
        if plan != "Executive Summary":
            print(f"\n{plan.upper()}")
            print("-" * 60)
            print(summary)
            print("\n")

    return final_state

# STEP 7: Interactive Chat for Follow-up Questions

The code block in Step 7 defines the chat_interface function, which provides an interactive chat for follow-up questions after the financial plan has been generated.

Here's a breakdown of its components and functionality:

* chat_interface(planning_state: AgentState) function:
  * Purpose: This function enables a conversational interaction with the user, allowing them to ask questions about their generated financial plan. The AI assistant uses the context of the complete financial plan to provide relevant and specific answers.
  * Input: It takes the planning_state (the final AgentState after the financial plan has been created) as input. This state contains all the user_info and plan_summaries.
  * Initialization: If planning_state is None (meaning no plan was generated), it exits. Otherwise, it sets up an introductory message and an empty conversation_history list to store previous exchanges.
  * Context Creation: It constructs a SystemMessage that serves as the AI's internal context. This message includes the user_info and all Financial Plan Summaries from the planning_state. This ensures the AI always has the complete financial picture when responding.
  * Interactive Loop: It enters a while True loop to allow continuous interaction:
    1. User Input: It prompts the user for a question. If the user types 'exit', 'quit', or 'bye', the chat ends.
    2. Message Construction: It builds a list of messages for the LLM, starting with the SystemMessage context, followed by the conversation_history, and finally the current HumanMessage from the user.
    3. LLM Invocation: It calls the llm.invoke() method with the constructed list of messages to get a response from the language model.
    4. History Update: The user's input and the AI's response are appended to the conversation_history to maintain conversational memory.
    5. Context Management: It keeps the conversation_history length manageable (e.g., last 10 messages) to prevent context window overflow and maintain efficiency.
    6. Display Response: The AI's response is printed to the console.

In [ ]:
def chat_interface(planning_state: AgentState):
    """Interactive chat interface for follow-up questions"""
    if planning_state is None:
        print("No planning session found. Please run the planning application first.")
        return

    print("\n" + "="*60)
    print("INTERACTIVE CHAT - Ask follow-up questions about your plan")
    print("Type 'exit' to quit")
    print("="*60 + "\n")

    conversation_history = []

    # Create context from planning state
    context = f"""You are a financial advisor assistant. You have access to the following information
about the client and their financial plan:

User Information:
{json.dumps(planning_state['user_info'], indent=2)}

Financial Plan Summaries:
{chr(10).join([f"### {plan}:{chr(10)}{summary}{chr(10)}" for plan, summary in planning_state['plan_summaries'].items()])}

Answer the user's questions based on this context. Be helpful, specific, and reference
the plans when relevant."""

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Thank you for using the Financial Planning Assistant!")
            break

        if not user_input:
            continue

        # Build conversation with context
        messages = [SystemMessage(content=context)]
        messages.extend(conversation_history)
        messages.append(HumanMessage(content=user_input))

        # Get response
        response = llm.invoke(messages)

        # Update conversation history
        conversation_history.append(HumanMessage(content=user_input))
        conversation_history.append(AIMessage(content=response.content))

        # Keep only last 10 messages to manage context
        if len(conversation_history) > 10:
            conversation_history = conversation_history[-10:]

        print(f"\nAssistant: {response.content}")

# RUN THE APPLICATION

In [ ]:
import sys
from pydantic import ValidationError # Import ValidationError

# Execute the planning application
print("Starting Financial Plan Summary Application...\n")

planning_result = None
while planning_result is None:
    try:
        planning_result = run_planning_application()
    except ValueError as e:
        print(f"\n⚠️  Input Error: {e}")
        print("Please ensure you provide valid numerical inputs when prompted. Let's try again.\n")
        if sys.stdin.isatty():
            sys.stdin.flush()
    except ValidationError as e: # Catch Pydantic ValidationErrors
        print(f"\n⚠️  Configuration Error in Agent Tool Call: {e}")
        print("The financial planning agent encountered an issue with its internal tool calls. ")
        print("It seems a required piece of user information was not passed correctly to a calculation tool. ")
        print("Please inform the developer of this issue to fix the agent's tool invocation logic.\n")
        break # Exit loop on configuration error, as user input won't fix it
    except Exception as e: # Catch any other unexpected errors
        print(f"\n⚠️  An unexpected error occurred: {type(e).__name__}: {e}")
        print("The planning application encountered an unhandled issue. Please review the details and try again.\n")
        break # Exit loop on unexpected error

# Start chat interface if planning was successful
if planning_result:
    print("\n" + "="*60)
    print("Planning complete! You can now ask follow-up questions.")
    print("="*60)
    chat_interface(planning_result)

Starting Financial Plan Summary Application...


FINANCIAL PLAN SUMMARY APPLICATION

Available Planning Options:
1. Retirement Planning
2. Insurance Planning
3. Estate Planning
4. Personal Wealth Management
5. Education Planning
6. Tax Planning

Enter plan numbers (comma-separated, e.g., 1,2,3,4,5,6): 1

Selected Plans: Retirement Planning

PLEASE PROVIDE YOUR INFORMATION
Your current age: 45
Annual income ($): 150000
Current savings/assets ($): 600000
Desired retirement age: 65
Risk tolerance (conservative/moderate/aggressive): moderate

GENERATING YOUR FINANCIAL PLAN...

📊 Using real-time year-over-year inflation rate: 2.39%
📊 Using real-time market data: S&P 500 change 0.05%

YOUR COMPREHENSIVE FINANCIAL PLAN

EXECUTIVE SUMMARY
------------------------------------------------------------
### Executive Summary

#### Client Overview
The client, aged 45, has an annual income of $150,000 and current savings of $600,000. With a retirement age set at 65, the client has 20 years until reti

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage | AIMessage | SystemMessage], operator.add]
    user_info: Dict[str, Any]
    selected_plans: List[str]
    plan_summaries: Dict[str, str]
    next_agent: str
    mcp_data: Dict[str, Any]

# STEP 8: Visualization Functions

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

def create_retirement_projection_chart(user_info: Dict[str, Any]):
    """Create retirement savings projection chart"""
    current_age = user_info.get('age', 30)
    retirement_age = user_info.get('retirement_age', 65)
    current_savings = user_info.get('savings', 0)
    annual_income = user_info.get('annual_income', 0)

    # Calculate monthly contribution needed
    years_to_retirement = retirement_age - current_age
    annual_expenses = annual_income * 0.8  # Assume 80% of income needed in retirement

    # Project savings with different contribution rates
    ages = list(range(current_age, 86))

    # Scenario 1: Current savings only (no additional contributions)
    no_contrib = []
    # Scenario 2: Conservative (5% of income)
    conservative = []
    # Scenario 3: Moderate (10% of income)
    moderate = []
    # Scenario 4: Aggressive (15% of income)
    aggressive = []

    for i, age in enumerate(ages):
        years_elapsed = i
        growth_rate = 0.07  # 7% annual return

        # No contributions
        no_contrib.append(current_savings * ((1 + growth_rate) ** years_elapsed))

        # With contributions (future value of annuity formula)
        if age < retirement_age:
            conservative_contrib = annual_income * 0.05
            moderate_contrib = annual_income * 0.10
            aggressive_contrib = annual_income * 0.15

            fv_contrib_c = conservative_contrib * (((1 + growth_rate) ** years_elapsed - 1) / growth_rate)
            fv_contrib_m = moderate_contrib * (((1 + growth_rate) ** years_elapsed - 1) / growth_rate)
            fv_contrib_a = aggressive_contrib * (((1 + growth_rate) ** years_elapsed - 1) / growth_rate)

            conservative.append(current_savings * ((1 + growth_rate) ** years_elapsed) + fv_contrib_c)
            moderate.append(current_savings * ((1 + growth_rate) ** years_elapsed) + fv_contrib_m)
            aggressive.append(current_savings * ((1 + growth_rate) ** years_elapsed) + fv_contrib_a)
        else:
            # After retirement, start drawing down
            withdrawal_rate = 0.04
            years_in_retirement = age - retirement_age

            conservative.append(max(0, conservative[-1] * ((1 + growth_rate - withdrawal_rate) ** 1)))
            moderate.append(max(0, moderate[-1] * ((1 + growth_rate - withdrawal_rate) ** 1)))
            aggressive.append(max(0, aggressive[-1] * ((1 + growth_rate - withdrawal_rate) ** 1)))

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=ages, y=no_contrib, name='No Additional Savings',
                             line=dict(color='red', width=2, dash='dash')))
    fig.add_trace(go.Scatter(x=ages, y=conservative, name='Conservative (5% savings)',
                             line=dict(color='orange', width=2)))
    fig.add_trace(go.Scatter(x=ages, y=moderate, name='Moderate (10% savings)',
                             line=dict(color='blue', width=3)))
    fig.add_trace(go.Scatter(x=ages, y=aggressive, name='Aggressive (15% savings)',
                             line=dict(color='green', width=2)))

    # Add retirement age line
    fig.add_vline(x=retirement_age, line_dash="dot", line_color="gray",
                  annotation_text="Retirement Age", annotation_position="top")

    fig.update_layout(
        title='Retirement Savings Projection',
        xaxis_title='Age',
        yaxis_title='Portfolio Value ($)',
        hovermode='x unified',
        template='plotly_white',
        height=500,
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
        yaxis=dict(tickformat='$,.0f')
    )

    return fig

def create_asset_allocation_pie(user_info: Dict[str, Any]):
    """Create asset allocation pie chart"""
    age = user_info.get('age', 30)
    risk_tolerance = user_info.get('risk_tolerance', 'moderate')
    total_assets = user_info.get('total_assets', user_info.get('savings', 0))

    # Calculate allocation
    base_stock_pct = 100 - age

    if risk_tolerance.lower() == "aggressive":
        stock_pct = min(90, base_stock_pct + 10)
    elif risk_tolerance.lower() == "conservative":
        stock_pct = max(20, base_stock_pct - 20)
    else:
        stock_pct = base_stock_pct

    bond_pct = 100 - stock_pct

    # Further breakdown
    allocations = {
        'US Stocks': stock_pct * 0.6,
        'International Stocks': stock_pct * 0.3,
        'Emerging Markets': stock_pct * 0.1,
        'Bonds': bond_pct * 0.7,
        'Cash/Money Market': bond_pct * 0.3
    }

    values = [total_assets * (pct / 100) for pct in allocations.values()]

    fig = go.Figure(data=[go.Pie(
        labels=list(allocations.keys()),
        values=values,
        hole=.3,
        marker=dict(colors=['#2E86AB', '#A23B72', '#F18F01', '#06A77D', '#D4AF37'])
    )])

    fig.update_layout(
        title=f'Recommended Asset Allocation ({risk_tolerance.title()} Profile)',
        annotations=[dict(text=f'${total_assets:,.0f}', x=0.5, y=0.5, font_size=20, showarrow=False)],
        height=500
    )

    return fig

def create_insurance_coverage_chart(user_info: Dict[str, Any]):
    """Create insurance coverage comparison chart"""
    annual_income = user_info.get('annual_income', 0)
    num_dependents = user_info.get('num_dependents', 0)
    debts = user_info.get('debts', 0)
    savings = user_info.get('savings', 0)

    # Calculate recommended coverage
    base_coverage = (annual_income * 10) + debts - savings
    dependent_factor = 1 + (num_dependents * 0.2)
    recommended_coverage = base_coverage * dependent_factor

    # Different insurance types
    insurance_types = ['Life Insurance', 'Disability Insurance', 'Critical Illness', 'Long-term Care']
    recommended = [
        recommended_coverage,
        annual_income * 5,  # Disability
        annual_income * 3,  # Critical illness
        300000  # Long-term care average
    ]

    current = [
        recommended_coverage * 0.4,  # Assume underinsured
        annual_income * 2,
        0,
        0
    ]

    fig = go.Figure()

    fig.add_trace(go.Bar(
        name='Current Coverage',
        x=insurance_types,
        y=current,
        marker_color='lightcoral'
    ))

    fig.add_trace(go.Bar(
        name='Recommended Coverage',
        x=insurance_types,
        y=recommended,
        marker_color='lightseagreen'
    ))

    fig.update_layout(
        title='Insurance Coverage Analysis',
        xaxis_title='Insurance Type',
        yaxis_title='Coverage Amount ($)',
        barmode='group',
        template='plotly_white',
        height=500,
        yaxis=dict(tickformat='$,.0f')
    )

    return fig

def create_education_funding_chart(user_info: Dict[str, Any]):
    """Create education funding progress chart"""
    num_children = user_info.get('num_children', 0)
    children_ages = user_info.get('children_ages', [])

    if num_children == 0 or not children_ages:
        return None

    cost_per_year = 30000
    inflation_rate = 0.05

    children_data = []
    for i, age in enumerate(children_ages):
        years_until_college = max(0, 18 - age)
        future_cost = cost_per_year * ((1 + inflation_rate) ** years_until_college)
        total_needed = future_cost * 4

        # Assume some savings already (30% of needed)
        current_savings = total_needed * 0.3

        children_data.append({
            'child': f'Child {i+1} (Age {age})',
            'needed': total_needed,
            'saved': current_savings,
            'gap': total_needed - current_savings,
            'years': years_until_college
        })

    fig = go.Figure()

    children_labels = [c['child'] for c in children_data]

    fig.add_trace(go.Bar(
        name='Amount Saved',
        x=children_labels,
        y=[c['saved'] for c in children_data],
        marker_color='mediumseagreen'
    ))

    fig.add_trace(go.Bar(
        name='Funding Gap',
        x=children_labels,
        y=[c['gap'] for c in children_data],
        marker_color='tomato'
    ))

    fig.update_layout(
        title='Education Funding Status',
        xaxis_title='Child',
        yaxis_title='Amount ($)',
        barmode='stack',
        template='plotly_white',
        height=500,
        yaxis=dict(tickformat='$,.0f')
    )

    return fig

def create_net_worth_projection(user_info: Dict[str, Any]):
    """Create net worth projection over time"""
    current_age = user_info.get('age', 30)
    annual_income = user_info.get('annual_income', 0)
    current_savings = user_info.get('savings', 0)
    debts = user_info.get('debts', 0)
    retirement_age = user_info.get('retirement_age', 65)

    ages = list(range(current_age, retirement_age + 30))

    assets = []
    liabilities = []
    net_worth = []

    annual_savings = annual_income * 0.15
    debt_payment = debts * 0.1 if debts > 0 else 0

    current_asset = current_savings
    current_debt = debts

    for i, age in enumerate(ages):
        growth_rate = 0.07

        if age < retirement_age:
            # Accumulation phase
            current_asset = current_asset * (1 + growth_rate) + annual_savings
            current_debt = max(0, current_debt - debt_payment)
        else:
            # Retirement phase
            current_asset = current_asset * (1 + growth_rate) - (annual_income * 0.8 * 0.04)
            current_debt = 0

        assets.append(current_asset)
        liabilities.append(current_debt)
        net_worth.append(current_asset - current_debt)

    fig = make_subplots(specs=[[{"secondary_y": False}]])

    fig.add_trace(go.Scatter(
        x=ages, y=assets, name='Assets',
        fill='tonexty',
        line=dict(color='green', width=2)
    ))

    fig.add_trace(go.Scatter(
        x=ages, y=liabilities, name='Liabilities',
        fill='tozeroy',
        line=dict(color='red', width=2)
    ))

    fig.add_trace(go.Scatter(
        x=ages, y=net_worth, name='Net Worth',
        line=dict(color='blue', width=3, dash='dash')
    ))

    fig.add_vline(x=retirement_age, line_dash="dot", line_color="gray",
                  annotation_text="Retirement", annotation_position="top")

    fig.update_layout(
        title='Net Worth Projection Over Time',
        xaxis_title='Age',
        yaxis_title='Amount ($)',
        hovermode='x unified',
        template='plotly_white',
        height=500,
        yaxis=dict(tickformat='$,.0f')
    )

    return fig

def create_monthly_budget_breakdown(user_info: Dict[str, Any]):
    """Create monthly budget breakdown"""
    annual_income = user_info.get('annual_income', 0)
    monthly_income = annual_income / 12

    # 50/30/20 rule with adjustments
    categories = {
        'Housing': monthly_income * 0.28,
        'Transportation': monthly_income * 0.15,
        'Food': monthly_income * 0.12,
        'Utilities': monthly_income * 0.08,
        'Insurance': monthly_income * 0.07,
        'Savings/Investments': monthly_income * 0.15,
        'Entertainment': monthly_income * 0.08,
        'Personal Care': monthly_income * 0.04,
        'Other': monthly_income * 0.03
    }

    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8',
              '#6C5CE7', '#FDCB6E', '#E17055', '#A29BFE']

    fig = go.Figure(data=[go.Bar(
        y=list(categories.keys()),
        x=list(categories.values()),
        orientation='h',
        marker=dict(color=colors),
        text=[f'${v:,.0f}' for v in categories.values()],
        textposition='auto',
    )])

    fig.update_layout(
        title=f'Recommended Monthly Budget (Total: ${monthly_income:,.0f})',
        xaxis_title='Monthly Amount ($)',
        yaxis_title='Category',
        template='plotly_white',
        height=500,
        showlegend=False,
        xaxis=dict(tickformat='$,.0f')
    )

    return fig

def display_all_visualizations(planning_state: AgentState):
    """Display all relevant visualizations based on selected plans"""
    if planning_state is None:
        print("No planning data available. Run the planning application first.")
        return

    user_info = planning_state['user_info']
    selected_plans = planning_state['selected_plans']

    print("\n" + "="*60)
    print("FINANCIAL PLAN VISUALIZATIONS")
    print("="*60 + "\n")

    # Net Worth Projection (always show)
    print("📊 Generating Net Worth Projection...")
    fig = create_net_worth_projection(user_info)
    fig.show()

    # Retirement Planning visualizations
    if "Retirement Planning" in selected_plans:
        print("\n📈 Generating Retirement Savings Projection...")
        fig = create_retirement_projection_chart(user_info)
        fig.show()

    # Wealth Management visualizations
    if "Personal Wealth Management" in selected_plans or "Retirement Planning" in selected_plans:
        print("\n🥧 Generating Asset Allocation Chart...")
        fig = create_asset_allocation_pie(user_info)
        fig.show()

        print("\n💰 Generating Monthly Budget Breakdown...")
        fig = create_monthly_budget_breakdown(user_info)
        fig.show()

    # Insurance Planning visualizations
    if "Insurance Planning" in selected_plans:
        print("\n🛡️ Generating Insurance Coverage Analysis...")
        fig = create_insurance_coverage_chart(user_info)
        fig.show()

    # Estate Planning visualizations
    if "Estate Planning" in selected_plans:
        num_children = user_info.get('num_children', 0)
        if num_children > 0:
            print("\n🎓 Generating Education Funding Status...")
            fig = create_education_funding_chart(user_info)
            if fig:
                fig.show()

    print("\n" + "="*60)
    print("All visualizations generated successfully!")
    print("="*60)

## Generate and display all visualizations

In [ ]:
display_all_visualizations(planning_result)


FINANCIAL PLAN VISUALIZATIONS

📊 Generating Net Worth Projection...



📈 Generating Retirement Savings Projection...



🥧 Generating Asset Allocation Chart...



💰 Generating Monthly Budget Breakdown...



All visualizations generated successfully!
